In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta

# 📁 Doelmap waar je wil downloaden
download_folder = r"D:\Data scientist\Jaar 1\Eindwerk\Test"

# ⚙️ Chrome instellingen om automatisch te downloaden
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
    "safebrowsing.enabled": False
}
chrome_options.add_experimental_option("prefs", prefs)

# 🧭 Start Chrome-driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 🌍 Bezoek de Elia website
driver.get("https://www.elia.be/nl/grid-data/productie/zonne-energieproductie")

import time
from selenium.common.exceptions import JavascriptException

# Geef de pagina tijd om de banner volledig te laden
time.sleep(5)

# Probeer "Ik weiger" aan te klikken via Shadow DOM
try:
    result = driver.execute_script("""
        const root = document.querySelector('#usercentrics-root');
        if (!root) return 'Root niet gevonden';

        const shadowRoot = root.shadowRoot;
        if (!shadowRoot) return 'Shadow root niet beschikbaar';

        const button = shadowRoot.querySelector('button[data-testid="uc-deny-all-button"]');
        if (!button) return 'Weiger-knop niet gevonden';

        button.click();
        return 'Klik op Ik weiger uitgevoerd';
    """)
    print("Cookiebanner resultaat:", result)
    time.sleep(2)
except JavascriptException as e:
    print("JavaScript-fout:", e)

# ⏳ Wachtinstelling
wait = WebDriverWait(driver, 20)

# 📅 Begin- en einddatum
start_date = datetime(2024, 1, 1)
end_date = datetime.now()

# 🔁 Loop door elke maand
current_date = start_date
while current_date <= end_date:
    # 👉 Formatteer zoals op de site, bv. "February 2024"
    maand_jaar = current_date.strftime('%B %Y')

    try:
        # Open de dropdown
        dropdown = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'elia-dropdown')))
        dropdown.click()

        # Wacht tot lijst geladen is
        opties = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'elia-dropdown__list-item')))

        # Zoek naar de juiste maand
        gevonden = False
        for optie in opties:
            if maand_jaar.lower() == optie.text.strip().lower():
                optie.click()
                gevonden = True
                break

        if not gevonden:
            print(f"Maand niet gevonden: {maand_jaar}")
            current_date += timedelta(days=31)
            current_date = current_date.replace(day=1)
            continue

    except Exception as e:
        print(f"Fout bij selecteren van {maand_jaar}: {e}")
        break

    # ⏳ Wacht even tot de data op de pagina geladen is
    time.sleep(5)

    try:
        # Klik op exportknop
        export_knop = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Export')]")))
        export_knop.click()
        print(f"Download gestart voor: {maand_jaar}")
    except Exception as e:
        print(f"Fout bij exporteren: {e}")
        break

    # ⌛ Tijd geven voor downloaden (je kan hier ook check op bestand doen als je wil)
    time.sleep(10)

    # Volgende maand
    current_date += timedelta(days=31)
    current_date = current_date.replace(day=1)

# ✅ Sluit browser
driver.quit()


Cookiebanner resultaat: Root niet gevonden
Fout bij selecteren van January 2024: Message: 
Stacktrace:
	GetHandleVerifier [0x006DC7F3+24435]
	(No symbol) [0x00662074]
	(No symbol) [0x005306E3]
	(No symbol) [0x00578B39]
	(No symbol) [0x00578E8B]
	(No symbol) [0x005C1AC2]
	(No symbol) [0x0059D804]
	(No symbol) [0x005BF20A]
	(No symbol) [0x0059D5B6]
	(No symbol) [0x0056C54F]
	(No symbol) [0x0056D894]
	GetHandleVerifier [0x009E70A3+3213347]
	GetHandleVerifier [0x009FB0C9+3295305]
	GetHandleVerifier [0x009F558C+3271948]
	GetHandleVerifier [0x00777360+658144]
	(No symbol) [0x0066B27D]
	(No symbol) [0x00668208]
	(No symbol) [0x006683A9]
	(No symbol) [0x0065AAC0]
	BaseThreadInitThunk [0x76195D49+25]
	RtlInitializeExceptionChain [0x776DCE3B+107]
	RtlGetAppContainerNamedObjectPath [0x776DCDC1+561]



In [ ]:
import requests
import pandas as pd
import os

# 📆 Begin- en einddatum in ISO-formaat (UTC)
start_date = "2024-01-01T00:00:00Z"
end_date = "2024-12-31T23:59:59Z"

# 📁 Lokale opslagmap
output_folder = r"D:\Data scientist\Jaar 1\Eindwerk\Test"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "SolarForecast_2024.csv")

# ✅ Correcte API endpoint
url = "https://opendata.elia.be/api/v2/datahub/forecasted_generation_solar_day_ahead/records"

# 📥 Parameters
params = {
    "where": f"datetime>='{start_date}' AND datetime<='{end_date}'",
    "limit": 10000
}

# 🚀 API-call
response = requests.get(url, params=params)

# ✅ Check status
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data["records"])
    df.to_csv(output_path, index=False)
    print(f"✅ Bestand opgeslagen als: {output_path}")
else:
    print(f"❌ Fout bij API-call. Status code: {response.status_code}")
    print("Antwoord:", response.text)


❌ Fout bij API-call. Status code: 404
Antwoord: {
  "error_code": "NotFoundURI",
  "message": "The requested URI does not exist. See /api/v2/swagger.json for a complete endpoints reference. Check the API documentation for more information: https://help.opendatasoft.com/apis/ods-explore-v2/."
}


In [ ]:
import requests
import pandas as pd
from datetime import datetime
import os

# 📆 Begin- en einddatum van je query
start_date = "2024-01-01T00:00:00Z"
end_date = "2024-12-31T23:59:59Z"

# 🗂️ Opslagpad
output_folder = r"D:\Data scientist\Jaar 1\Eindwerk\Test"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "SolarForecast_2024.csv")

# 🔗 API URL en parameters
url = "https://opendata.elia.be/api/v2/datahub/forecasted_generation_solar_day_ahead/json"

params = {
    "start": start_date,
    "end": end_date,
    "limit": 10000  # maximaal aantal rijen
}

# 🚀 API-call
response = requests.get(url, params=params)
data = response.json()

# 🧾 Omzetten naar dataframe
df = pd.DataFrame(data['results'])

# 💾 Opslaan naar CSV
df.to_csv(output_path, index=False)
print(f"Bestand opgeslagen als: {output_path}")


KeyboardInterrupt: 

In [ ]:
import requests
import os

# Map en bestandsnaam voor opslag
output_folder = r"Test"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "SolarForecast_Elia_20250227.json")

# Download JSON-dataset van Elia
url = "https://opendata.elia.be/api/explore/v2.1/catalog/datasets/ods032/records"
params = {
    "order_by": "datetime",
    "limit": 100,
    "refine": [
        'datetime:"2025/02/27"',
        'region:"Belgium"'
    ]
}

response = requests.get(url, params=params)

# Check status
if response.status_code == 200:
    data = response.json()
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print("✔ JSON-bestand opgeslagen.")

In [ ]:
import requests
import os
import pandas as pd

# 📁 Map en bestandsnaam voor opslag
# output_folder = r"D:\Data scientist\Jaar 1\Eindwerk\Test"
output_folder = r"Test"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "SolarForecast_Elia_20250227.json")
filtered_csv_path = os.path.join(output_folder, "SolarForecast_2024.csv")

# 🔗 Download JSON-dataset van Elia
# url = "https://opendata.elia.be/explore/dataset/ods032/download?format=json"
# url = "https://opendata.elia.be/api/explore/v2.1/catalog/datasets/ods032/records?order_by=datetime&limit=100&refine=datetime%3A%222025%2F02%2F27%22&refine=region%3A%22Belgium%22"
# response = requests.get(url)

url = "https://opendata.elia.be/api/explore/v2.1/catalog/datasets/ods032/records"
params = {
    "order_by": "datetime",
    "limit": 100,
    "refine": [
        'datetime:"2025/02/27"',
        'region:"Belgium"'
    ]
}

response = requests.get(url, params=params)

# ✅ Check status
if response.status_code == 200:
    data = response.json()
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print("✔ JSON-bestand opgeslagen.")
"""
    # 🧾 Data converteren naar DataFrame
    records = [entry['fields'] for entry in data if 'fields' in entry]
    df = pd.DataFrame(records)

    # 🔍 Filter op datums vanaf 2024
    df['datetime'] = pd.to_datetime(df['datetime'])
    df_filtered = df[df['datetime'] >= "2024-01-01"]

    # 💾 Opslaan als CSV
    df_filtered.to_csv(filtered_csv_path, index=False)
    print(f"✔ Gefilterde data opgeslagen in: {filtered_csv_path}")
else:
    print(f"❌ Fout bij downloaden: {response.status_code}")
"""

✔ JSON-bestand opgeslagen.


'\n    # 🧾 Data converteren naar DataFrame\n    records = [entry[\'fields\'] for entry in data if \'fields\' in entry]\n    df = pd.DataFrame(records)\n\n    # 🔍 Filter op datums vanaf 2024\n    df[\'datetime\'] = pd.to_datetime(df[\'datetime\'])\n    df_filtered = df[df[\'datetime\'] >= "2024-01-01"]\n\n    # 💾 Opslaan als CSV\n    df_filtered.to_csv(filtered_csv_path, index=False)\n    print(f"✔ Gefilterde data opgeslagen in: {filtered_csv_path}")\nelse:\n    print(f"❌ Fout bij downloaden: {response.status_code}")\n'

In [ ]:
import json
import pandas as pd

# JSON-bestand inladen
with open(r'Import\WindForecast\WindForecast_Elia_20250201.json') as f:
    data = json.load(f)

# Als het een lijst van records is:
df = pd.DataFrame(data)

# Opslaan naar Excel
df.to_csv(r'Import\WindForecast\output.csv', index=False)

## SolarForecast eerste versie

## WindForecast eerste versie

In [ ]:
# Jaar en maand die je wil gebruiken
year = 2025
month = 2

# Bepaal het aantal dagen in de maand
_, num_days = calendar.monthrange(year, month)

# Map voor opslag
output_folder = r"Import\WindForecast"
os.makedirs(output_folder, exist_ok=True)

# Loop over elke dag van de maand
for day in range(1, num_days + 1):
    # Bouw datumstring in formaat "YYYY/MM/DD"
    date_str = f"{year}/{month:02d}/{day:02d}"
    output_filename = f"WindForecast_Elia_{year}{month:02d}{day:02d}.json"
    output_path = os.path.join(output_folder, output_filename)

    # Controleer of bestand al bestaat
    if os.path.exists(output_path):
        print(f"⏭ Bestand bestaat al: {output_filename}")
        continue  # Ga naar volgende dag

    # Stel de API-parameters in
    url = "https://opendata.elia.be/api/explore/v2.1/catalog/datasets/ods031/records"
    params = {
        "order_by": "datetime",
        "limit": 100,
        "refine": [
            f'datetime:"{date_str}"',
            'region:"Flanders"'
        ]
    }

    # Verstuur de aanvraag
    response = requests.get(url, params=params)

    # Check status en sla op
    if response.status_code == 200:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(response.text)
        print(f"✔ Opgeslagen: {output_filename}")
    else:
        print(f"✖ Fout bij {date_str}: {response.status_code}")

⏭ Bestand bestaat al: WindForecast_Elia_20250201.json
⏭ Bestand bestaat al: WindForecast_Elia_20250202.json
⏭ Bestand bestaat al: WindForecast_Elia_20250203.json
⏭ Bestand bestaat al: WindForecast_Elia_20250204.json
⏭ Bestand bestaat al: WindForecast_Elia_20250205.json
⏭ Bestand bestaat al: WindForecast_Elia_20250206.json
⏭ Bestand bestaat al: WindForecast_Elia_20250207.json
⏭ Bestand bestaat al: WindForecast_Elia_20250208.json
⏭ Bestand bestaat al: WindForecast_Elia_20250209.json
⏭ Bestand bestaat al: WindForecast_Elia_20250210.json
⏭ Bestand bestaat al: WindForecast_Elia_20250211.json
⏭ Bestand bestaat al: WindForecast_Elia_20250212.json
⏭ Bestand bestaat al: WindForecast_Elia_20250213.json
⏭ Bestand bestaat al: WindForecast_Elia_20250214.json
⏭ Bestand bestaat al: WindForecast_Elia_20250215.json
⏭ Bestand bestaat al: WindForecast_Elia_20250216.json
⏭ Bestand bestaat al: WindForecast_Elia_20250217.json
⏭ Bestand bestaat al: WindForecast_Elia_20250218.json
⏭ Bestand bestaat al: WindFo

In [ ]:
# Jaar en maand die je wil gebruiken
year = 2025
month = 2

# Bepaal het aantal dagen in de maand
_, num_days = calendar.monthrange(year, month)

# Map voor opslag
output_folder = r"Import\SolarForecast"
os.makedirs(output_folder, exist_ok=True)

# Loop over elke dag van de maand
for day in range(1, num_days + 1):
    # Bouw datumstring in formaat "YYYY/MM/DD"
    date_str = f"{year}/{month:02d}/{day:02d}"
    output_filename = f"SolarForecast_Elia_{year}{month:02d}{day:02d}.json"
    output_path = os.path.join(output_folder, output_filename)

    # Controleer of bestand al bestaat
    if os.path.exists(output_path):
        print(f"⏭ Bestand bestaat al: {output_filename}")
        continue  # Ga naar volgende dag

    # Stel de API-parameters in
    url = "https://opendata.elia.be/api/explore/v2.1/catalog/datasets/ods032/records"
    params = {
        "order_by": "datetime",
        "limit": 100,
        "refine": [
            f'datetime:"{date_str}"',
            'region:"Belgium"'
        ]
    }

    # Verstuur de aanvraag
    response = requests.get(url, params=params)

    # Check status en sla op
    if response.status_code == 200:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(response.text)
        print(f"✔ Opgeslagen: {output_filename}")
    else:
        print(f"✖ Fout bij {date_str}: {response.status_code}")


⏭ Bestand bestaat al: SolarForecast_Elia_20250201.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250202.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250203.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250204.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250205.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250206.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250207.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250208.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250209.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250210.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250211.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250212.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250213.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250214.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250215.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250216.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250217.json
⏭ Bestand bestaat al: SolarForecast_Elia_20250218.json
⏭ Bestand 

## Import van Belpex info via webscraping

### Werkt gedeeltelijk

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

# Instellingen voor de datums
from_date = "01/04/2024"
until_date = "02/04/2024"

# Setup voor Chrome, zonder headless mode (zodat je de browser kunt zien)
options = Options()
# options.add_argument("--headless")  # Niet headless, zodat we kunnen zien wat er gebeurt
driver = webdriver.Chrome(options=options)

# Ga naar de website
driver.get("https://my.elexys.be/MarketInformation/SpotBelpex.aspx")

# Wacht tot het formulier en de knop "Show data" beschikbaar zijn
wait = WebDriverWait(driver, 20)
try:
    wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_fromASPxDateEdit_I")))
    print("✅ From-date veld gevonden!")
except:
    print("❌ From-date veld niet gevonden!")

try:
    wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_refreshBelpexCustomButton_I")))
    print("✅ Show data knop gevonden!")
except:
    print("❌ Show data knop niet gevonden!")

# Vul de datums in het formulier in
from_input = driver.find_element(By.ID, "contentPlaceHolder_fromASPxDateEdit_I")
until_input = driver.find_element(By.ID, "contentPlaceHolder_untilASPxDateEdit_I")

print(f"⚙️ Vul 'From' datum in: {from_date}")
from_input.clear()
from_input.send_keys(from_date)

print(f"⚙️ Vul 'Until' datum in: {until_date}")
until_input.clear()
until_input.send_keys(until_date)

# Zoekknop vinden en klik via JavaScript
show_data_button = driver.find_element(By.ID, "contentPlaceHolder_refreshBelpexCustomButton_I")
print("⚙️ Klik op 'Show data' knop met JavaScript")
driver.execute_script("arguments[0].click();", show_data_button)

# Wacht tot de resultaten geladen zijn (verhoog tijd voor stabiliteit)
print("⏳ Wacht op zoekresultaten...")
wait.until(EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder1_gvBelpex")))
time.sleep(5)  # Extra wachttijd voor stabiliteit

# Zoek de exporteer knop
export_csv_button = driver.find_element(By.ID, "ctl00_contentPlaceHolder_GridViewExportUserControl1_csvExport_I")
print("⚙️ Klik op 'Exporteer naar CSV' knop met JavaScript")

# Gebruik JavaScript om te klikken
driver.execute_script("arguments[0].click();", export_csv_button)

# Wacht tot het CSV-bestand is gedownload
time.sleep(5)  # Pas de wachttijd aan afhankelijk van hoe snel het bestand wordt gedownload

# Sluit de browser af
driver.quit()

✅ From-date veld gevonden!
✅ Show data knop gevonden!
⚙️ Vul 'From' datum in: 01/04/2024
⚙️ Vul 'Until' datum in: 02/04/2024
⚙️ Klik op 'Show data' knop met JavaScript
⏳ Wacht op zoekresultaten...


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7DEA45335+78597]
	GetHandleVerifier [0x00007FF7DEA45390+78688]
	(No symbol) [0x00007FF7DE7F91AA]
	(No symbol) [0x00007FF7DE84F149]
	(No symbol) [0x00007FF7DE84F3FC]
	(No symbol) [0x00007FF7DE8A2467]
	(No symbol) [0x00007FF7DE87712F]
	(No symbol) [0x00007FF7DE89F2BB]
	(No symbol) [0x00007FF7DE876EC3]
	(No symbol) [0x00007FF7DE8403F8]
	(No symbol) [0x00007FF7DE841163]
	GetHandleVerifier [0x00007FF7DECEEEED+2870973]
	GetHandleVerifier [0x00007FF7DECE9698+2848360]
	GetHandleVerifier [0x00007FF7DED06973+2967875]
	GetHandleVerifier [0x00007FF7DEA6017A+188746]
	GetHandleVerifier [0x00007FF7DEA6845F+222255]
	GetHandleVerifier [0x00007FF7DEA4D2B4+111236]
	GetHandleVerifier [0x00007FF7DEA4D462+111666]
	GetHandleVerifier [0x00007FF7DEA33589+5465]
	BaseThreadInitThunk [0x00007FF9E81CE8D7+23]
	RtlUserThreadStart [0x00007FF9E8CB14FC+44]


### Werkt

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

# Instellingen voor de datums
from_date = "01/01/2024"
until_date = "31/12/2024"

# Downloadpad instellen
download_dir = os.path.join(os.getcwd(), "Belpex")
os.makedirs(download_dir, exist_ok=True)

# Setup voor Chrome
options = Options()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)

# Ga naar de website
driver.get("https://my.elexys.be/MarketInformation/SpotBelpex.aspx")

# Wacht tot het formulier en de knop "Show data" beschikbaar zijn
wait = WebDriverWait(driver, 20)
wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_fromASPxDateEdit_I")))
wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_refreshBelpexCustomButton_I")))

# Vul de datums in
from_input = driver.find_element(By.ID, "contentPlaceHolder_fromASPxDateEdit_I")
until_input = driver.find_element(By.ID, "contentPlaceHolder_untilASPxDateEdit_I")

print(f"⚙️ Vul 'From' datum in: {from_date}")
from_input.clear()
from_input.send_keys(from_date)

print(f"⚙️ Vul 'Until' datum in: {until_date}")
until_input.clear()
until_input.send_keys(until_date)

# Klik op "Show data"
show_data_button = driver.find_element(By.ID, "contentPlaceHolder_refreshBelpexCustomButton_I")
print("⚙️ Klik op 'Show data'")
driver.execute_script("arguments[0].click();", show_data_button)

# Wacht op tabelresultaten
print("⏳ Wacht op zoekresultaten...")
wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_belpexFilterGrid_DXMainTable")))
time.sleep(5)  # Extra wachttijd voor stabiliteit

# ✅ Klik op de juiste export-div
print("⚙️ Klik op 'Exporteer naar CSV'")
export_button_div = wait.until(EC.element_to_be_clickable((By.ID, "ctl00_contentPlaceHolder_GridViewExportUserControl1_csvExport")))
driver.execute_script("arguments[0].click();", export_button_div)

# Wacht op de download
print("⏳ Wacht op download...")
time.sleep(5)

# Check of het CSV-bestand werd gedownload
files = os.listdir(download_dir)
csvs = [f for f in files if f.endswith(".csv")]
if csvs:
    print(f"✅ CSV gedownload: {csvs[-1]}")
else:
    print("❌ Geen CSV-bestand gevonden.")

# Sluit de browser
driver.quit()

⚙️ Vul 'From' datum in: 01/01/2024
⚙️ Vul 'Until' datum in: 31/12/2024
⚙️ Klik op 'Show data'
⏳ Wacht op zoekresultaten...
⚙️ Klik op 'Exporteer naar CSV'
⏳ Wacht op download...
✅ CSV gedownload: BelpexFilter.csv


### Met download per maand

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from datetime import datetime
import calendar

# Instellingen voor het jaar en de maand (bijvoorbeeld 2024, januari)
year = 2024
month = 1  # Dit kan elke maand zijn van 1 tot 12

# Bereken de datums voor de maand
from_date = f"01/{month:02d}/{year}"  # Eerste dag van de maand

# Bereken de laatste dag van de maand
last_day = calendar.monthrange(year, month)[1]  # Haal het aantal dagen van de maand op
until_date = f"{last_day:02d}/{month:02d}/{year}"  # Laatste dag van de maand

# Downloadpad instellen
download_dir = os.path.join(os.getcwd(), "Belpex")
os.makedirs(download_dir, exist_ok=True)

# Setup voor Chrome
options = Options()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)

# Ga naar de website
driver.get("https://my.elexys.be/MarketInformation/SpotBelpex.aspx")

# Wacht tot het formulier en de knop "Show data" beschikbaar zijn
wait = WebDriverWait(driver, 20)
wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_fromASPxDateEdit_I")))
wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_refreshBelpexCustomButton_I")))

# Vul de datums in
from_input = driver.find_element(By.ID, "contentPlaceHolder_fromASPxDateEdit_I")
until_input = driver.find_element(By.ID, "contentPlaceHolder_untilASPxDateEdit_I")

print(f"⚙️ Vul 'From' datum in: {from_date}")
from_input.clear()
from_input.send_keys(from_date)

print(f"⚙️ Vul 'Until' datum in: {until_date}")
until_input.clear()
until_input.send_keys(until_date)

# Klik op "Show data"
show_data_button = driver.find_element(By.ID, "contentPlaceHolder_refreshBelpexCustomButton_I")
print("⚙️ Klik op 'Show data'")
driver.execute_script("arguments[0].click();", show_data_button)

# Wacht op tabelresultaten
print("⏳ Wacht op zoekresultaten...")
wait.until(EC.presence_of_element_located((By.ID, "contentPlaceHolder_belpexFilterGrid_DXMainTable")))
time.sleep(5)  # Extra wachttijd voor stabiliteit

# ✅ Klik op de juiste export-div
print("⚙️ Klik op 'Exporteer naar CSV'")
export_button_div = wait.until(EC.element_to_be_clickable((By.ID, "ctl00_contentPlaceHolder_GridViewExportUserControl1_csvExport")))
driver.execute_script("arguments[0].click();", export_button_div)

# Wacht op de download
print("⏳ Wacht op download...")
time.sleep(5)

# Check of het CSV-bestand werd gedownload
files = os.listdir(download_dir)
csvs = [f for f in files if f.endswith(".csv")]

# Herbenoem het bestand naar 'Belpex_JJJJMM.csv' (bijv. Belpex_202401.csv)
if csvs:
    downloaded_file = csvs[-1]
    new_filename = f"Belpex_{year}{month:02d}.csv"
    os.rename(os.path.join(download_dir, downloaded_file), os.path.join(download_dir, new_filename))
    print(f"✅ CSV gedownload en hernoemd naar: {new_filename}")
else:
    print("❌ Geen CSV-bestand gevonden.")

# Sluit de browser
driver.quit()

⚙️ Vul 'From' datum in: 01/01/2024
⚙️ Vul 'Until' datum in: 01/02/2024
⚙️ Klik op 'Show data'
⏳ Wacht op zoekresultaten...
⚙️ Klik op 'Exporteer naar CSV'
⏳ Wacht op download...
✅ CSV gedownload en hernoemd naar: Belpex_202401.csv
